26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: マークアップ早見表）．
https://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8

In [0]:
from google.colab import files
import pandas as pd
import io
import re

#Pandasの表示を100文字に広げる
pd.set_option("display.max_colwidth", 100)

In [2]:
uploaded = files.upload()

Saving jawiki-country.json.gz to jawiki-country.json.gz


In [0]:
def extract_by_title(title):
    df_wiki = pd.read_json('jawiki-country.json.gz', compression='infer', lines=True)
    return df_wiki[ (df_wiki['title'] == title)]['text'].values[0]

In [0]:
wiki_body = extract_by_title('イギリス')

In [0]:
#3重クォートとre.VERBOSEオプションを使うことによって、空白無視
#re.MULTILINEで複数行に対して検索
pattern_basic = re.compile(r'''
                           ^\{\{基礎情報\s.*?$  #検索語句(\はエスケープ処理, \sはスペース)
                           (.*?)                #任意の文字列
                           \}\}                 #検索語句(\はエスケープ処理)
                           $                    #文字列の末尾
                           ''',re.MULTILINE + re.VERBOSE  + re.DOTALL)

result_basic = pattern_basic.findall(wiki_body)

In [12]:
removal_emphasys = re.compile(r'''
                             ''       #'を2つ検索
                             ('*)     #'が連続であるかを検索 -> Group1(\1)にしておく
                             (.+)     #"任意文字列(強調箇所)-> Group2(\2)にしておく
                             ''       #'を2つ検索
                             \1       #Group1部分('の追加連続部分)
                             ''',re.MULTILINE + re.VERBOSE )
result_basic[0] = removal_emphasys.sub(r"\2", result_basic[0]) #Group2強調箇所で置換

print(result_basic)

['\n|略名 = イギリス\n|日本語国名 = グレートブリテン及び北アイルランド連合王国\n|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>\n*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（[[スコットランド・ゲール語]]）<br/>\n*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（[[ウェールズ語]]）<br/>\n*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（[[アイルランド語]]）<br/>\n*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（[[コーンウォール語]]）<br/>\n*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（[[スコットランド語]]）<br/>\n**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>\n|国旗画像 = Flag of the United Kingdom.svg\n|国章画像 = [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]\n|国章リンク = （[[イギリスの国章|国章]]）\n|標語 = {{lang|fr|Dieu et mon droit}}<br/>（[[フランス語]]:神と私の権利）\n|国歌 = [[女王陛下万歳|神よ女王陛下を守り給え]]\n|位置画像 = Location_UK_EU_Euro

In [13]:
removal_inlink = re.compile(r'''
                             \[\[        #[2つをエスケープ処理
                             (?:         #対象外グループ開始
                               [^|\]]*?  # '|'または]以外の文字が0文字以上、非貪欲
                               \|        # '|'
                             )?          # グループ終了、このグループが0か1出現
                             ([^|]*?)    # 検索語句、'|'以外が0文字以上、非貪欲（Group1）
                             \]\]
                             ''',re.MULTILINE + re.VERBOSE )
result_basic[0] = removal_inlink.sub(r"\1", result_basic[0]) #Group1リンク記事名で置換

print(result_basic)

['\n|略名 = イギリス\n|日本語国名 = グレートブリテン及び北アイルランド連合王国\n|公式国名 = {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>\n*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（スコットランド・ゲール語）<br/>\n*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（ウェールズ語）<br/>\n*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（アイルランド語）<br/>\n*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（コーンウォール語）<br/>\n*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（スコットランド語）<br/>\n**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>\n|国旗画像 = Flag of the United Kingdom.svg\n|国章画像 = [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]\n|国章リンク = （国章）\n|標語 = {{lang|fr|Dieu et mon droit}}<br/>（フランス語:神と私の権利）\n|国歌 = 神よ女王陛下を守り給え\n|位置画像 = Location_UK_EU_Europe_001.svg\n|公用語 = 英語（事実上）\n|首都 = ロンドン\n|最大都市 =

In [14]:
pattern_line = re.compile(r'''
                          ^\|         #検索語句(\はエスケープ処理)
                          (.+?)       #任意の文字列
                          (?:         # キャプチャ対象外のグループ開始
                            (?=\n\|)  # 改行(\n)+'|'の手前（先読みアサーション）
                          | (?=\n$)   # または、改行(\n)+終端の手前（先読みアサーション）
                          )           # グループ終了
                           ''',re.MULTILINE + re.VERBOSE  + re.DOTALL)
result_line = pattern_line.findall(result_basic[0])

basic_dict = {}

for line in result_line:
    #=で分割してDictionary作成
    basic_info_list = line.split('=')
    basic_dict[basic_info_list[0]] = basic_info_list[1]
    #print(basic_info_list[0],basic_info_list[1])

for key, value in basic_dict.items():
    print(key, value)

略名   イギリス
日本語国名   グレートブリテン及び北アイルランド連合王国
公式国名   {{lang|en|United Kingdom of Great Britain and Northern Ireland}}<ref>英語以外での正式国名:<br/>
*{{lang|gd|An Rìoghachd Aonaichte na Breatainn Mhòr agus Eirinn mu Thuath}}（スコットランド・ゲール語）<br/>
*{{lang|cy|Teyrnas Gyfunol Prydain Fawr a Gogledd Iwerddon}}（ウェールズ語）<br/>
*{{lang|ga|Ríocht Aontaithe na Breataine Móire agus Tuaisceart na hÉireann}}（アイルランド語）<br/>
*{{lang|kw|An Rywvaneth Unys a Vreten Veur hag Iwerdhon Glédh}}（コーンウォール語）<br/>
*{{lang|sco|Unitit Kinrick o Great Breetain an Northren Ireland}}（スコットランド語）<br/>
**{{lang|sco|Claught Kängrick o Docht Brätain an Norlin Airlann}}、{{lang|sco|Unitet Kängdom o Great Brittain an Norlin Airlann}}（アルスター・スコットランド語）</ref>
国旗画像   Flag of the United Kingdom.svg
国章画像   [[ファイル:Royal Coat of Arms of the United Kingdom.svg|85px|イギリスの国章]]
国章リンク   （国章）
標語   {{lang|fr|Dieu et mon droit}}<br/>（フランス語:神と私の権利）
国歌   神よ女王陛下を守り給え
位置画像   Location_UK_EU_Europe_001.svg
公用語   英語（事実上）
首都   ロンドン
最大都市   ロンドン
元首等肩書   女王
元首等氏名   エリザベス2世
